In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown


### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

**Interpolation problem with cubic Spline** - fulfilling grid function $y_{i}$ = $f(x_{i}), i = \overline{0,n}, x_{i}\in[a, b]$ using Cubic Splines $S_{3}(x)\in C_{2}[a, b]$ 

$S_{3,i}(x) = a_{0,i} + a_{1,i}(x - x_{i}) + a_{2,i}(x - x_{i})^2 + a_{3,i}(x - x_{i})^3$ 




### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

###Functional condition: 

$\delta S_{3,i}(x)|_{x=x_i} = S_{3,i}(x_{i})-f(x_i)=0$

$\delta S_{3,i}(x)|_{x=x_{i+1}} = S_{3,i}(x_{i+1})-f(x_i+1)=0$


###Differential condition: 

$\delta S_{3,i}''(x)|_{x=x_i} = S_{3,i}''(x_{i})-f''(x_i)=0$

$\delta S_{3,i}''(x)|_{x=x_{i+1}} = S_{3,i}''(x_{i+1})-f''(x_i+1)=0$

### 3. Formulate stitching conditions [mathematical formula]

$S_{3,i-1}'(x)|_{x=x_i}=S_{3,i}'(x)|_{x=x_i}$

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

**The functional condition** provides a solution to the interpolation problem. ($C_0$)

**The differential condition** provides a continuity to a solution.($C_1$)

**The stitching condition** provides smootheness to a solution.($C_2$)

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

$S_{3,i}(x) = a_{0,i} + a_{1,i}(x - x_{i}) + a_{2,i}(x - x_{i})^2 + a_{3,i}(x - x_{i})^3$ 

$S_{3,i}'(x) = a_{1,i} + 2a_{2,i}(x - x_{i}) + 3a_{3,i}(x - x_{i})^2$

$S_{3,i}''(x) = 2a_{2,i} + 6a_{3,i}(x - x_{i})$

Using functional and differential conditions for coefficients of polynom:

$a_{0,i} - f_i = 0 $    

$a_{0,i}+a_{1,i}(x_{i+1} - x_i) + a_{2,i}(x_{i+1} - x_i)^2+a_{3,i}(x_{i+1} - x_i)^3-f_{i+1}=0      $

$2a_{2,i}-f''_i=0 $

$2a_{2,i}+6a_{3,i}(x_{i+1}-x_i)-f''_{i+1}=0$


Let:

$m_i=f''_i$

$h_{i+1}=x_{i+1}-x_i$

$\delta f_i = f_{i+1}-f_i$

$\Delta m_i = m_{i+1}-m_i$

So,

$S_{3,i}(x)=f_i+(\frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i)(x-x_i) + \frac{m_i}{2}(x-x_i)^2+ \frac{1}{6h_{i+1}} \Delta m_i(x-x_i)^3$


$\frac{ \Delta f_i - \Delta f_{i-1}}{h} = \frac{h}{6}m_{i-1} + \frac{2h}{3}m_i + \frac{h}{6}m_{i+1}, i = \overline{1,n-1}$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

$
\left( \begin{array}{ccc}
\frac{h}{6} & \frac{2h}{3} & \frac{h}{6} & 0 & 0 & \ldots\\
0 & \frac{h}{6} & \frac{2h}{3} & \frac{h}{6} & 0 & \ldots\\
0 & 0 & \frac{h}{6} & \frac{2h}{3} & \frac{h}{6} & \ldots\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & \ldots & \frac{2h}{3} & \frac{h}{6} & \frac{2h}{3}
\end{array} \right)
$
$
\left( \begin{array}{ccc}
m_1\\
m_2\\
\vdots\\
m_{n-1}
\end{array} \right)
$
$=$
$\left( \begin{array}{ccc}
\frac{\Delta f_1 - \Delta f_0}{h}\\
\frac{\Delta f_2 - \Delta f_1}{h}\\
\vdots\\
\frac{\Delta f_{n-1} - \Delta f_{n-2}}{h}
\end{array} \right)$

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

System is not closed because $m_i$ ranges from 0 to n, while we have only from 1 to n-1, so we need 2 more equations to solve

**Solution:**  $m_0 = m_n = 0$ (assigning second derivatives on ends as 0)


### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$\left(\begin{array}{ccc}
\frac{2h}{3} & \frac{h}{6} & 0 & 0 & \ldots & \frac{\Delta f_1 - \Delta f_0}{h}\\
\frac{h}{6} & \frac{2h}{3} & \frac{h}{6} & 0 & \ldots & \frac{\Delta f_2 - \Delta f_1}{h}\\
 0 & \frac{h}{6} & \frac{2h}{3} & \frac{h}{6} & \ldots  & \frac{\Delta f_3 - \Delta f_2}{h}\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & \ldots & \frac{2h}{3} & \frac{h}{6} & \frac{\Delta f_{n-1} - \Delta f_{n-2}}{h}
\end{array} \right)$

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Direct pass: 

$P_i = \frac{\gamma_i}{\beta_i - \alpha_i P_{i-1}}, \ Q_i = \frac{\alpha_i Q_{i-1} - \delta_i}{\beta_i - \alpha_i P_{i-1}}, \ i = \overline{1,n-1}$

Reverse pass: 

$x_n = \frac{\alpha_n Q_{n-1} - \delta_n}{\beta_n - \alpha_n P_{i-1}}$

### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [0]:
def tridiagonal
  pass
def calc_coeffs
  pass
def spline
  pass


### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

$||f^{(p)}(x) - S^{(p)}_3(x)||_{C[a,b]} = \max_{[a,b]}|f^{(p)}(x)-S^{(p)}_3(x)| \leq M_4 h^{}$

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

**Direct pass** - O(N)

**Reverse pass** - O(N)

**Total:** O(N)

### Congrats!